In [1]:
import pandas as pd
import numpy as np
import cohere 
import sys, os
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from co_key import *


In [2]:
train_data = pd.read_json('../data/relations_dev.json')
test_data = pd.read_json('../data/relations_test.json')

In [3]:
train_data.head()

,document,tokens,relations
0,Bachelor's degree in Mechanical Engineering or...,"[{'text': 'Bachelor', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'DEG..."
1,10+ years of software engineering work experie...,"[{'text': '10+ years', 'start': 0, 'end': 9, '...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP..."
2,3+ years Swift & Objective-C and experience wi...,"[{'text': '3+ years', 'start': 0, 'end': 8, 't...","[{'child': 3, 'head': 0, 'relationLabel': 'EXP..."
3,8+ years experience in software engineering le...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 5, 'head': 0, 'relationLabel': 'EXP..."
4,BS degree in Computer Science or related field...,"[{'text': 'BS', 'start': 0, 'end': 2, 'token_s...","[{'child': 3, 'head': 0, 'relationLabel': 'DEG..."


In [4]:
train = '../data/cleaned_train_job_data.json'
test = '../data/cleaned_test_job_data.json'

In [5]:
import json


with open (train, 'r') as f:
    train_clean = json.loads(f.read())
with open (test , 'r') as f:
    test_clean = json.loads(f.read())

In [6]:
train_clean[0]

{'document': "Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience",
 'tokens': [{'text': 'Bachelor', 'entityLabel': 'DIPLOMA'},
  {'text': 'Mechanical Engineering', 'entityLabel': 'DIPLOMA_MAJOR'},
  {'text': 'Physical Science', 'entityLabel': 'DIPLOMA_MAJOR'},
  {'text': '3+ years', 'entityLabel': 'EXPERIENCE'},
  {'text': 'developing', 'entityLabel': 'SKILLS'},
 

In [7]:
co = cohere.Client(x)


In [8]:
class Promptor:
    def __init__(self) -> None:
        pass    
    def extract_values(self, data):
            #extract the values from each json dictionaries and convert them to a prompt types
        s = ''
        for d in data:
            s+='Job Description: '
            s+=str(d['document']).strip().replace('\n', ' ').strip()
            s+='\n'
            
                #Extract entities from tokens
            token_dict = {}
            for token in d['tokens']:
                entity_label = token['entityLabel']
                if entity_label in token_dict.keys():
                    token_dict[entity_label] += f",{token['text']}"
                else:
                    token_dict[entity_label] = f"{token['text']}"
            dict_str = ''
            for key,value in token_dict.items():
                dict_str+=f"{key}: {value}\n"
            s+=dict_str
            s+='\n--\n'
        return s
prompting = Promptor()


In [9]:
values = prompting.extract_values(train_clean)
with open('../data/train_finetune.txt','w') as f:
    f.write(values)

In [10]:
value = prompting.extract_values(test_clean)
with open('../data/test_finetune.txt','w') as f:
    f.write(value)

In [12]:
val = prompting.extract_values(train_clean[:5])